# Unit 4: People Perception Part 1. Face Detection and tracking

<img src="img/perception_unit4_peopleintro.png"/>

<img src="img/rosject.png" width="250" style="float:left;" />

* ROSject Link: http://bit.ly/2LPKOGv

* Package Name: **face_detection_tc**

* Launch File: **main.launch**

Another vital skill that a robot has to have is the ability to work alongside humans. This means that it has to detect when a human is around, address it by looking into its eyes, and be able to recognise it.

Basically, there are three main skills concerning humans that are basic for a robot to have:

* **Detect Faces**: This means that it can detect where a human face is and address it for talking or giving feedback. But it can't know who it is, it just knows that it's a human face. This is important for emotion reading, listening to orders, or just giving certain feedback that is so necessary for Human Robot Interaction, or HRI.
* **Recognise Faces**: Now this skill allows the robot to know who he is talking to. This is vital for security purposes or just to know how to address each individual.
* **Track People**: The robot has to be able to track the movements of humans, be able to follow the one it needs to follow, and filter the others, without colliding with them.

These three basic skills will be addressed in the next Units.<br>
In this Unit, you will learn how to detect faces.

## Face Detector in ROS

You will use the package <a href="http://wiki.ros.org/face_detector#Example_with_robots">face_detector</a>.

### Starting the Face Detector Server

The first step is to start the face detector server. For this, you will have to create your new package called **my_face_detector**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd;cd ../src
catkin_create_pkg my_face_detector rospy face_detector

Inside its launch folder, create a launch file called, for example, **face_detection_tc.launch**. Here you have the contents of that file:

<p style="background:green;color:white;">**face_detection_tc.launch**</p>

In [ ]:
<launch>

    <arg name="camera" default="head_camera" />
    <arg name="depth_ns" default="depth_registered" />
    <arg name="image_topic" default="image_raw" />
    <arg name="depth_topic" default="image_raw" />
    <arg name="fixed_frame" default="head_camera_rgb_optical_frame" />
    <arg name="rgb_ns" default="rgb" />
    
    <!--
    DEAPTH ==> /head_camera/depth_registered/points or /head_camera/depth_registered/image_raw
    RGB ==> /head_camera/rgb/image_raw
    rostopic echo /face_detector/people_tracker_measurements_array
    rosnode info /face_detector
    -->
    
    <!--include file="$(find openni_launch)/launch/openni.launch"/-->
    <!--
    <node name="$(anon dynparam)" pkg="dynamic_reconfigure" type="dynparam" args="set_from_parameters $(arg camera)/driver">
    <param name="depth_registration" type="bool" value="true" />
    </node>
    -->
    
    
    <node pkg="face_detector" type="face_detector" name="face_detector" output="screen">
        <remap from="camera" to="$(arg camera)" />
        <remap from="image_topic" to="$(arg image_topic)" />
        <remap from="depth_topic" to="$(arg depth_topic)" />
        <remap from="depth_ns" to="$(arg depth_ns)" />
        <remap from="rgb_ns" to="$(arg rgb_ns)" />
        <param name="fixed_frame" type="string" value="$(arg fixed_frame)" />
        
        <param name="classifier_name" type="string" value="frontalface" />
        <rosparam command="load" file="$(find face_detector)/param/classifier.yaml"/>
        <param name="classifier_reliability" type="double" value="0.9"/>
        <param name="do_continuous" type="bool" value="true" />
        <param name="do_publish_faces_of_unknown_size" type="bool" value="false" /> 
        <param name="do_display" type="bool" value="false" />
        <param name="use_rgbd" type="bool" value="true" />
        <param name="approximate_sync" type="bool" value="true" />
    </node>
    
</launch>

<p style="background:green;color:white;">**END face_detection_tc.launch**</p>

There are many things to comment on here:

In [ ]:
<arg name="camera" default="head_camera" />
<arg name="depth_ns" default="depth_registered" />
<arg name="image_topic" default="image_raw" />
<arg name="depth_topic" default="image_raw" />
<arg name="fixed_frame" default="head_camera_rgb_optical_frame" />
<arg name="rgb_ns" default="rgb" />

The setting of the arguments is vital for this to work. It's divided into basic elements of the topics so that it can use all of the topics that you have from the PointCloud Camera.

The face detection uses two types of data:<br>

* RGB image: This is published, in this case, in the topic **/head_camera/rgb/image_raw**
* Depth Image Data: This is published in the topic **/head_camera/depth_registered/image_raw**

These topics, then, have to be divided into the corresponding arguments, resulting in setting them the way mentioned.<br>
The **fixed_frame** selected is the one where the PCL camera is. In this case, it's the **head_camera_rgb_optical_frame**. Normally, it's always the optical frame of the robot that is selected.

<p style="color:green">**Note that because you are using a simulation, you don't need to start the openni server, it's already done for you. In the case of the real robot, you will probably have to start it.**</p>

In [ ]:
<!--include file="$(find openni_launch)/launch/openni.launch"/-->
<!--
<node name="$(anon dynparam)" pkg="dynamic_reconfigure" type="dynparam" args="set_from_parameters $(arg camera)/driver">
<param name="depth_registration" type="bool" value="true" />
</node>
-->

And now, the start of the face_detector:

In [ ]:
<node pkg="face_detector" type="face_detector" name="face_detector" output="screen">
    <remap from="camera" to="$(arg camera)" />
    <remap from="image_topic" to="$(arg image_topic)" />
    <remap from="depth_topic" to="$(arg depth_topic)" />
    <remap from="depth_ns" to="$(arg depth_ns)" />
    <remap from="rgb_ns" to="$(arg rgb_ns)" />
    <param name="fixed_frame" type="string" value="$(arg fixed_frame)" />

    <param name="classifier_name" type="string" value="frontalface" />
    <rosparam command="load" file="$(find face_detector)/param/classifier.yaml"/>
    <param name="classifier_reliability" type="double" value="0.9"/>
    <param name="do_continuous" type="bool" value="true" />
    <param name="do_publish_faces_of_unknown_size" type="bool" value="false" /> 
    <param name="do_display" type="bool" value="false" />
    <param name="use_rgbd" type="bool" value="true" />
    <param name="approximate_sync" type="bool" value="true" />
</node>

Here you set the arguments and then you set some other parameters. You can leave them as they are to detect faces that are facing forward. For more details, refer to: http://wiki.ros.org/face_detector

<p style="background:#EE9023;color:white;">**Exercise U4-1**</p>

Create a package named **my_face_detector**, where you have the above launch file. Then, execute it and see what happens, what new topics are published, and what exactly they are publishing.

<p style="background:#EE9023;color:white;">**END Exercise U4-1**</p>

So, as you might have already seen, although you launch the system, there is no detection. There is nothing really published. That's because the server will only publish if there is a client connected to it. This is quite common in well-designed servers to avoid overflooding the ROS system with data that no one is listening to.

In [ ]:
[ INFO] [1501668161.272087967, 446.303000000]: You must subscribe to one of FaceDetector's outbound topics or else it will not publish anything.

So, obviously, the next step is creating a client to trigger the face_detection.

### Face Detector Client

First, you will have to add to the basic **face_detection_tc.launch** the following line at the end, in order to launch your client:

In [ ]:
<!-- The  face_detector needs for someone to subscribe to publish data and detect people's faces -->
<node pkg="my_face_detector" type="face_detector_client.py" name="face_detection_client_start_node" output="screen">
</node>

Create in the **scripts** folder of the **my_face_detector** package, the following Python file named **face_detector_client.py**.

<p style="background:green;color:white;">**face_detector_client.py**</p>

In [ ]:
#!/usr/bin/env python

import rospy
from people_msgs.msg import PositionMeasurementArray

# Move base using navigation stack
class FaceDetectClient(object):

    def __init__(self):
        self.face_detect_subs = rospy.Subscriber(   "/face_detector/people_tracker_measurements_array",
                                                    PositionMeasurementArray,
                                                    self.face_detect_subs_callback)

    
        self.pos_mesurement_array = PositionMeasurementArray()
    
    def face_detect_subs_callback(self,msg):
        self.pos_mesurement_array = msg

    
def Face_DetectionClient_Start():
    # Create a node
    rospy.init_node("face_detection_client_start_node")

    # Make sure sim time is working
    while not rospy.Time.now():
        pass

    face_detector_client = FaceDetectClient()

    rospy.spin()

if __name__ == "__main__":
    Face_DetectionClient_Start()



<p style="background:green;color:white;">**END face_detector_client.py**</p>

As you can see, there is no mystery here. You just subscribe to the topic, **/face_detector/people_tracker_measurements_array**, and all the face detecting topics will have to start publishing data. Let's have a look at this data:

In [ ]:
user catkin_ws $ rosmsg show people_msgs/PositionMeasurementArray                                              
std_msgs/Header header                                               
  uint32 seq                                               
  time stamp                                               
  string frame_id                                              
people_msgs/PositionMeasurement[] people                                               
  std_msgs/Header header                                               
    uint32 seq                                               
    time stamp                                               
    string frame_id                                              
  string name                                              
  string object_id                                               
  geometry_msgs/Point pos                                              
    float64 x                                              
    float64 y                                              
    float64 z                                              
  float64 reliability                                              
  float64[9] covariance                                              
  byte initialization                                              
float32[] cooccurrence

As you can see, you will get the position of every face detected, with an ID for each one. Perfect for tracking many people at the same time.

<p style="background:#EE9023;color:white;">**Exercise U4-2**</p>

As explained right above, create this new Python script and launch it alongside the face_detector node. See how data starts pouring in.

If everything works fine, you should get in the output something similar to this:

In [ ]:
...
[ INFO] [1517390863.451964928, 708.716000000]: Number of faces found: 1, number with good depth and size: 1
[ INFO] [1517390863.943576089, 708.852000000]: Detection duration = 0.136809sec
[ INFO] [1517390863.943895491, 708.852000000]: Found face to match with id 0
[ INFO] [1517390863.943990702, 708.852000000]: Number of faces found: 1, number with good depth and size: 1
[ INFO] [1517390864.183386652, 708.874000000]: Detection duration = 0.134366sec
[ INFO] [1517390864.183462408, 708.874000000]: Found face to match with id 0
...

<p style="background:#EE9023;color:white;">**END Exercise U4-2**</p>

## Visualise the Face detections

As you may have already guessed, in perception, visualising the detection data is crucial in order to understand if the robot is really comprehending what's going on around it. That's why you have to use RViz and special markers to indicate where the face detections are made and when they stop.

For this, you will need to copy a premade RViz config file that we provide:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd face_detection_tc
cp -r rviz_config /home/user/catkin_ws/src/my_face_detector/rviz_config

Now, launch RViz and **open this new RViz config file**. You should see something similar to this:

<span style="color:red;">**NOTE**: For this to work properly, you need to have the launch file executed in **ExerciseU4-2** running and working as expected.

<img src="img/perception_unit4_facedetectrviz2.png"/>

This topic representation <i>( **/face_detector/people_tracker_measurements_array** )</i> is giving you the position of the face detected. As you can see, it is placed more or less where the person frame TF is.<br>
Let's have a look at the topics read for getting this information, so that you can reproduce this anywhere:


* The first thing to note is that this is not your run-of-the-mill RViz marker. This is because it's using the <a href="http://jsk-visualization.readthedocs.io/en/latest/jsk_rviz_plugins/index.html">jsk_rviz_plugins</a> package. You will have to install this package if you want to use it outside the Ignite Academy or RDS environments. These markers give a lot of functionality to RViz and you could spend a whole course learning all the places where this can be applied. You can have a look at the <a href="https://www.robotigniteacademy.com/en/course/ros-rviz-advanced-markers/details/" target="_blank">ROS RViz Advanced Markers</a> course to learn more about RViz Markers.
* The LaserScan and the PointCloud2 data are disconnected to avoid overflooding the PC, because PCL consumes quite a lot of resources. But you can activate them by simply checking the box.
* **PeoplePositionMesurementsArray**: This is the blue circle drawn around the position of the face detected. It's the main data we are looking for.
* Camera: Just the RGB camera, as a reference, which RViz superimposes the PeoplePositionMesurementsArray data.


<p style="background:#EE9023;color:white;">**Exercise U4-3**</p>

Move the Fetch robot and the person around to see if the tracking is correct and coincides with the real positions.<br>
For this, you will have to:<br>

* Create a Python script that moves the Fetch robot around. For that, you will have to publish into the **/base_controller/command** topic, using Twist messages.
* Create another Python script that moves the person around. For that, you will have to publish into the **/person_standing/cmd_vel** topic, using Twist messages. You can use the launch that is already done to see how it moves:<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch person_sim move_person_standing.launch



* Create a new Fetch moving script that moves the Fetch robot to try and follow the person moving around. If it loses the face, create a recovery behavior. Try not to use the TF data of the person because that would be cheating, as you are not going to have that data in real life. You will only have the TF data of your own robot and the tranformation from your TF to the PositionMesurement of the face detection.


<p style="background:#EE9023;color:white;">**END Exercise U4-3**</p>

<p style="background:#417FB1;color:white;">**Project**</p>

You can now do the 4th exercise of the Aibo Project. There, you will have to make the Aibo robot search for human faces, get to their position, and when Aibo is close enough (around 1 meter), wag its tail. Then, go to the next human and repeat the process. 

<p style="background:#417FB1;color:white;">**END Project**</p>